<a href="https://colab.research.google.com/github/aquibjaved/Bits_and_Pieces_DL/blob/tinytransformer/Basic_trfs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
!pip install transformers datasets tokenizers

In [3]:
import torch
import torch.nn as nn
import math

from datasets import load_dataset
from transformers import AutoTokenizer
from torch.utils.data import DataLoader

In [4]:
class InputEmbeddings(nn.Module):
  def __init__(self, model_dim:int=512, vocab_size:int=30000):
    super().__init__()

    self.model_dim = model_dim
    self.vocab_size = vocab_size

    self.embedding = nn.Embedding(num_embeddings=self.vocab_size, embedding_dim=self.model_dim)

  def forward(self, x):
    return self.embedding(x) * math.sqrt(self.model_dim)


In [5]:
class PositionalEncoding(nn.Module):
  def __init__(self, model_dim:int, seq_len:int, dropout:float):
    super().__init__()

    self.model_dim = model_dim
    self.seq_len= seq_len # max len of doc
    self.dropout = nn.Dropout(dropout)

    # create matrix of shape (seq, model_dim) => (512, 512)
    pe = torch.zeros(seq_len, model_dim)

    # create a vector for seq_len
    position = torch.arange(0, self.seq_len, dtype=torch.float).unsqueeze(1)
    div_term = torch.exp(torch.arange(0, model_dim, 2).float() * (-math.log(10000.0)/self.model_dim))
    pe[:, 0::2] = torch.sin(position * div_term)
    pe[:, 1::2] = torch.cos(position * div_term)

    pe = pe.unsqueeze(0)

    self.register_buffer('pe', pe)

  def forward(self, x):
    x = x + (self.pe[:, :x.shape[1], :]).requires_grad_(False)
    return self.dropout(x)

In [6]:
PE = PositionalEncoding(model_dim=512, seq_len=512, dropout=0.0)

In [11]:
class LayerNormalization(nn.Module):
  def __init__(self, eps: float=10**-6):
    super().__init__()
    self.eps = eps
    self.alpha = nn.Parameter(torch.ones(1))
    self.bias = nn.Parameter(torch.zeros(1))

  def forward(self, x):
    mean = x.mean(dim = -1, keepdim=True)
    std = x.std(dim = -1, keepdim=True)

    return self.alpha * (x-mean)/(std+self.eps) + self.bias


In [12]:
torch.zeros(1)

tensor([0.])

In [226]:
class FeedForwardBlock(nn.Module):
  def __init__(self, model_dim:int, d_ff:int, dropout:float):
    super().__init__()

    self.d_ff = d_ff
    self.linear_one = nn.Linear(model_dim, self.d_ff)
    self.dropout = nn.Dropout(dropout)
    self.linear_out = nn.Linear(self.d_ff, model_dim)


  def forward(self, x):
    # (batch_size, max_seq_len, model_dim)

    return self.linear_out(self.dropout(torch.relu(self.linear_one(x))))


In [227]:
class ResidualConnection(nn.Module):

        def __init__(self, features: int, dropout: float) -> None:
            super().__init__()
            self.dropout = nn.Dropout(dropout)
            self.norm = LayerNormalization(features)

        def forward(self, x, sublayer):
            return x + self.dropout(sublayer(self.norm(x)))

In [228]:
class MultiHeadAttentionBlock(nn.Module):
  def __init__(self, h:int=8, model_dim:int=512, dropout:float=0.1):
    super().__init__()

    self.model_dim = model_dim
    self.h = h
    self.dropout = nn.Dropout(dropout)

    assert model_dim % h == 0
    self.d_k = model_dim//h

    # now get the weights of query, key, value
    self.w_q = nn.Linear(model_dim, model_dim)
    self.w_k = nn.Linear(model_dim, model_dim)
    self.w_v = nn.Linear(model_dim, model_dim)

    self.w_o = nn.Linear(model_dim, model_dim)

    self.dropout = nn.Dropout(dropout)

  @staticmethod
  def attention(query, key, value, mask, dropout: nn.Dropout):
    d_k = query.shape[-1]
    # Just apply the formula from the paper
    # (batch, h, seq_len, d_k) --> (batch, h, seq_len, seq_len)
    attention_scores = (query @ key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        # Write a very low value (indicating -inf) to the positions where mask == 0
        attention_scores.masked_fill_(mask == 0, -1e9)

    attention_scores = attention_scores.softmax(dim=-1) # (batch, h, seq_len, seq_len) # Apply softmax

    if dropout is not None:
        attention_scores = dropout(attention_scores)
    # (batch, h, seq_len, seq_len) --> (batch, h, seq_len, d_k)
    # return attention scores which can be used for visualization
    return (attention_scores @ value), attention_scores

  def forward(self, q, k, v, mask):

    query = self.w_q(q)
    key = self.w_k(k)
    value = self.w_v(v)

    query = query.view(query.shape[0], query.shape[1], self.h, self.d_k).transpose(1,2)
    key = value.view(key.shape[0], key.shape[1], self.h, self.d_k).transpose(1,2)
    value = value.view(value.shape[0], value.shape[1], self.h, self.d_k).transpose(1,2)

    # calculate attention
    x, self.attention_score = MultiHeadAttentionBlock.attention(query, key, value, mask, self.dropout)

    # (batch, h, seq_len, d_k) --> (batch, seq_len, h, d_k) --> (batch, seq_len, model_dim)
    x = x.transpose(1, 2).contiguous().view(x.shape[0], -1, self.h * self.d_k)

    # Multiply by Wo
    # (batch, seq_len, model_dim) --> (batch, seq_len, model_dim)
    return self.w_o(x)

In [229]:
class EncoderBlock(nn.Module):

    def __init__(self, features: int, self_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForwardBlock, dropout: float) -> None:
        super().__init__()
        self.self_attention_block = self_attention_block
        self.feed_forward_block = feed_forward_block
        self.residual_connections = nn.ModuleList([ResidualConnection(features, dropout) for _ in range(2)])

    def forward(self, x, src_mask):
        x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, src_mask))
        x = self.residual_connections[1](x, self.feed_forward_block)
        return x

In [250]:
class Encoder(nn.Module):
  def __init__(self, features:int, layers:nn.ModuleList):
    super().__init__()
    self.layers = layers
    self.norm = LayerNormalization(features)

  def forward(self, x, mask):
    for layer in self.layers:
      x = layer(x, mask)
    return self.norm(x)

In [251]:
class DecoderBlock(nn.Module):

    def __init__(self, features: int, self_attention_block: MultiHeadAttentionBlock, cross_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForwardBlock, dropout: float) -> None:
        super().__init__()
        self.self_attention_block = self_attention_block
        self.cross_attention_block = cross_attention_block
        self.feed_forward_block = feed_forward_block
        self.residual_connections = nn.ModuleList([ResidualConnection(features, dropout) for _ in range(3)])

    def forward(self, x, encoder_output, src_mask, tgt_mask):
        x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, tgt_mask))
        x = self.residual_connections[1](x, lambda x: self.cross_attention_block(x, encoder_output, encoder_output, src_mask))
        x = self.residual_connections[2](x, self.feed_forward_block)
        return x

In [252]:
class Decoder(nn.Module):

    def __init__(self, features: int, layers: nn.ModuleList) -> None:
        super().__init__()
        self.layers = layers
        self.norm = LayerNormalization(features)

    def forward(self, x, encoder_output, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, encoder_output, src_mask, tgt_mask)
        return self.norm(x)

In [253]:
class ProjectionLayer(nn.Module):
  def __init__(self, model_dims:int, vocab_size:int):
    super().__init__()

    self.model_dims = model_dims
    self.vocab_size = vocab_size

    self.proj = nn.Linear(self.model_dims, self.vocab_size)

  def forward(self, x):
    return torch.log_softmax(self.proj(x), dim =-1)

In [254]:
class Transformer(nn.Module):

    def __init__(self, encoder: Encoder, decoder: Decoder, src_embed: InputEmbeddings, tgt_embed: InputEmbeddings, src_pos: PositionalEncoding, tgt_pos: PositionalEncoding, projection_layer: ProjectionLayer) -> None:
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.src_pos = src_pos
        self.tgt_pos = tgt_pos
        self.projection_layer = projection_layer

    def encode(self, src, src_mask):
        # (batch, seq_len, model_dim)
        src = self.src_embed(src)
        src = self.src_pos(src)
        return self.encoder(src, src_mask)

    def decode(self, encoder_output: torch.Tensor, src_mask: torch.Tensor, tgt: torch.Tensor, tgt_mask: torch.Tensor):
        # (batch, seq_len, model_dim)
        tgt = self.tgt_embed(tgt)
        tgt = self.tgt_pos(tgt)
        return self.decoder(tgt, encoder_output, src_mask, tgt_mask)

    def project(self, x):
        # (batch, seq_len, vocab_size)
        return self.projection_layer(x)




In [255]:
def build_transformer(src_vocab_size: int, tgt_vocab_size: int, src_seq_len: int, tgt_seq_len: int, model_dim: int=512, N: int=6, h: int=8, dropout: float=0.1, d_ff: int=2048) -> Transformer:
    # Create the embedding layers
    src_embed = InputEmbeddings(model_dim, src_vocab_size)
    tgt_embed = InputEmbeddings(model_dim, tgt_vocab_size)

    # Create the positional encoding layers
    src_pos = PositionalEncoding(model_dim, src_seq_len, dropout)
    tgt_pos = PositionalEncoding(model_dim, tgt_seq_len, dropout)

    # Create the encoder blocks
    encoder_blocks = []
    for _ in range(N):
        encoder_self_attention_block = MultiHeadAttentionBlock(model_dim, h, dropout)
        feed_forward_block = FeedForwardBlock(model_dim, d_ff, dropout)
        encoder_block = EncoderBlock(model_dim, encoder_self_attention_block, feed_forward_block, dropout)
        encoder_blocks.append(encoder_block)

    # Create the decoder blocks
    decoder_blocks = []
    for _ in range(N):
        decoder_self_attention_block = MultiHeadAttentionBlock(model_dim, h, dropout)
        decoder_cross_attention_block = MultiHeadAttentionBlock(model_dim, h, dropout)
        feed_forward_block = FeedForwardBlock(model_dim, d_ff, dropout)
        decoder_block = DecoderBlock(model_dim, decoder_self_attention_block, decoder_cross_attention_block, feed_forward_block, dropout)
        decoder_blocks.append(decoder_block)

    # Create the encoder and decoder
    encoder = Encoder(model_dim, nn.ModuleList(encoder_blocks))
    decoder = Decoder(model_dim, nn.ModuleList(decoder_blocks))

    # Create the projection layer
    projection_layer = ProjectionLayer(model_dim, tgt_vocab_size)

    # Create the transformer
    transformer = Transformer(encoder, decoder, src_embed, tgt_embed, src_pos, tgt_pos, projection_layer)

    # Initialize the parameters
    for p in transformer.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)

    return transformer

In [256]:
model_dim = 512
src_vocab_size = 30522
tgt_vocab_size = 30522
src_seq_len = 512
tgt_seq_len = 512
dropout = 0.1
h = 8
N = 6
d_ff = 2048

In [257]:


src_embed = InputEmbeddings(model_dim, src_vocab_size)
tgt_embed = InputEmbeddings(model_dim, tgt_vocab_size)

# Create the positional encoding layers
src_pos = PositionalEncoding(model_dim, src_seq_len, dropout)
tgt_pos = PositionalEncoding(model_dim, tgt_seq_len, dropout)


# Create the encoder blocks
encoder_blocks = []
for _ in range(N):
    encoder_self_attention_block = MultiHeadAttentionBlock(h, model_dim, dropout)
    feed_forward_block = FeedForwardBlock(model_dim, d_ff, dropout)
    encoder_block = EncoderBlock(model_dim, encoder_self_attention_block, feed_forward_block, dropout)
    encoder_blocks.append(encoder_block)


# Create the decoder blocks
decoder_blocks = []
for _ in range(N):
    decoder_self_attention_block = MultiHeadAttentionBlock(h, model_dim, dropout)
    decoder_cross_attention_block = MultiHeadAttentionBlock(h, model_dim, dropout)
    feed_forward_block = FeedForwardBlock(model_dim, d_ff, dropout)
    decoder_block = DecoderBlock(model_dim, decoder_self_attention_block, decoder_cross_attention_block, feed_forward_block, dropout)
    decoder_blocks.append(decoder_block)


# Create the encoder and decoder
encoder = Encoder(model_dim, nn.ModuleList(encoder_blocks))
decoder = Decoder(model_dim, nn.ModuleList(decoder_blocks))



# Create the projection layer
projection_layer = ProjectionLayer(model_dim, tgt_vocab_size)

# Create the transformer
transformer = Transformer(encoder, decoder, src_embed, tgt_embed, src_pos, tgt_pos, projection_layer)


In [258]:
from datasets import load_dataset

dataset = load_dataset("findnitai/english-to-hinglish", split="train")

In [259]:
dataset['translation'][0]

{'en': "What's the name of the movie",
 'hi_ng': 'film ka kya naam hai',
 'source': 1}

In [260]:
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer

class BilingualDataset(Dataset):

    def __init__(self, ds, model_name_src, model_name_tgt, src_lang, tgt_lang, seq_len):
        super().__init__()
        self.seq_len = seq_len

        self.ds = ds
        # Load the tokenizers
        self.tokenizer_src = AutoTokenizer.from_pretrained(model_name_src)
        self.tokenizer_tgt = AutoTokenizer.from_pretrained(model_name_tgt)
        self.src_lang = src_lang
        self.tgt_lang = tgt_lang

        # Assuming that [SOS] and [EOS] tokens are already in the tokenizer's vocabulary
        self.sos_token_id = self.tokenizer_tgt.convert_tokens_to_ids('[SOS]')
        self.eos_token_id = self.tokenizer_tgt.convert_tokens_to_ids('[EOS]')
        self.pad_token_id = self.tokenizer_tgt.pad_token_id

    def __len__(self):
        return len(self.ds)

    def __getitem__(self, idx):
        src_target_pair = self.ds[idx]
        src_text = src_target_pair['translation'][self.src_lang]
        tgt_text = src_target_pair['translation'][self.tgt_lang]

        # Encode the texts
        encoder_input = self.tokenizer_src.encode(src_text, max_length=self.seq_len, padding='max_length', truncation=True)
        decoder_input = [self.sos_token_id] + self.tokenizer_tgt.encode(tgt_text, max_length=self.seq_len - 1, truncation=True)

        # Create the label by shifting the decoder_input and adding [EOS] token
        label = decoder_input[1:] + [self.eos_token_id]

        # Ensure the length is seq_len
        decoder_input = decoder_input + [self.pad_token_id] * (self.seq_len - len(decoder_input))
        label = label + [self.pad_token_id] * (self.seq_len - len(label))

        encoder_input = torch.tensor(encoder_input, dtype=torch.long)
        decoder_input = torch.tensor(decoder_input, dtype=torch.long)
        label = torch.tensor(label, dtype=torch.long)

        return {
            "encoder_input": encoder_input,
            "encoder_mask": (encoder_input != self.pad_token_id).unsqueeze(0).unsqueeze(0).int(),

            "decoder_input": decoder_input,
            "decoder_mask": (decoder_input != self.pad_token_id).unsqueeze(0).int() & causal_mask(decoder_input.size(0)), # (1, seq_len) & (1, seq_len, seq_len),

            "label": label,
            "src_text": src_text,
            "tgt_text": tgt_text,
        }

# Function to create the causal mask for the decoder
def causal_mask(size):
    mask = torch.triu(torch.ones((1, size, size)), diagonal=1).bool()
    return ~mask


In [261]:
from transformers import AutoTokenizer

# Load the pre-trained tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [262]:
hinglish_ds = BilingualDataset(ds=dataset, model_name_src="bert-base-uncased", model_name_tgt="bert-base-uncased", src_lang="en", tgt_lang="hi_ng", seq_len=512)

In [263]:
hinglish_dataloader = DataLoader(hinglish_ds, batch_size=4)

In [264]:
data = {}
for batch in hinglish_dataloader:
  data = batch
  break

In [265]:
data.get('encoder_input')[0]

tensor([ 101, 2054, 1005, 1055, 1996, 2171, 1997, 1996, 3185,  102,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,   

In [266]:
data.get('encoder_mask')[0]

tensor([[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [268]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
optimizer = torch.optim.Adam(transformer.parameters(), lr =0.0001, eps=1e-9)
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=tokenizer.convert_tokens_to_ids(['[PAD]']), label_smoothing=0.1)

transformer.train()
for batch in hinglish_dataloader:

  encoder_input = batch['encoder_input']
  encoder_mask = batch['encoder_mask']
  decoder_input = batch['decoder_input']
  decoder_mask = batch['decoder_mask']

  enocder_output = transformer.encode(encoder_input, encoder_mask)
  print(enocder_output)


  break


tensor([[[ 0.0400,  0.0965,  0.0899,  ..., -0.0014, -0.0197,  0.0332],
         [-0.0368, -0.0552,  0.0361,  ...,  0.0917, -0.0147, -0.0035],
         [-0.0758, -0.0183,  0.0037,  ...,  0.0688, -0.0022,  0.0049],
         ...,
         [-0.0453, -0.0034,  0.0420,  ...,  0.0318, -0.0764, -0.0036],
         [-0.0422,  0.0275,  0.0437,  ...,  0.0333, -0.0750, -0.0520],
         [-0.0028,  0.0244,  0.0412,  ...,  0.0323, -0.0761, -0.0530]],

        [[ 0.0398,  0.0967,  0.0900,  ...,  0.0207, -0.0196,  0.0334],
         [ 0.0489, -0.0008, -0.0026,  ..., -0.0079,  0.0439, -0.0785],
         [-0.0428, -0.0130,  0.0975,  ...,  0.1170, -0.0022, -0.0051],
         ...,
         [-0.0447,  0.0277, -0.0029,  ...,  0.0324, -0.0756, -0.0030],
         [-0.0435,  0.0263,  0.0426,  ...,  0.0321, -0.0760, -0.0531],
         [-0.0441,  0.0235,  0.0403,  ...,  0.0313, -0.0767, -0.0538]],

        [[ 0.0408,  0.0975,  0.0910,  ...,  0.0214, -0.0187, -0.0006],
         [ 0.0219, -0.0042, -0.0461,  ...,  0